In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import copy
from evaluation import score
from sksurv.preprocessing import OneHotEncoder
from sksurv.metrics import concordance_index_censored as Cindex
import re

In [2]:
data_path_intro = '../data/data_dictionary.csv'
df_vars = pd.read_csv(data_path_intro)

# create dict to specify data type
type_mapping = {'Categorical':'category', 'Numerical':'Float64'}
data_type_dict = {df_vars['variable'][i]:type_mapping[df_vars['type'][i]] for i in range(len(df_vars['variable']))}

# read training and testing data
data_path_train = '../data/train.csv'
df_train = pd.read_csv(data_path_train, dtype=data_type_dict)

data_path_test = '../data/test.csv'
df_test = pd.read_csv(data_path_test, dtype=data_type_dict)

data_path_sample = '../data/sample_submission.csv'
df_sample = pd.read_csv(data_path_sample)

df_train_data = df_train.drop(columns=['efs','efs_time', 'ID'])
df_train_target = df_train[['efs', 'efs_time']]

# techniques: 
<br>
1 imputation of numerical values
<br>
2 age < 20 and age >= 20
<br>
4 2020 and 2019 combined

In [3]:
# create embedding for each type category
cat_inds = {}
cat_var_list = []
num_var_list = []
num_var_exp_list = []
for i in range(df_vars.shape[0]):
    tp_var = df_vars['variable'][i]
    if data_type_dict[tp_var] == 'category' and tp_var!='efs':
        tp_cats = re.findall(r"'(.*?)'", df_vars['values'][i])
        cat_inds[tp_var] = {tp_cats[i]:i+1 for i in range(len(tp_cats))}
        cat_var_list.append(tp_var)
    elif tp_var != 'efs_time' and tp_var!='efs':
        num_var_list.append(tp_var)
        num_var_exp_list.append(df_vars['description'][i])
        
# add additional to cat_inds
for i in range(df_vars.shape[0]):
    tp_var = df_vars['variable'][i]
    if tp_var not in cat_inds and tp_var != 'donor_age' and tp_var != 'age_at_hct' and tp_var!='efs' and tp_var!='efs_time':
        # get all values from the table
        values_all = [tp_val for tp_val in  list(set(df_train_data[tp_var].to_list())) if tp_val is not pd.NA]
        values_all.sort()
        cat_inds[tp_var] = {values_all[i]:i+1 for i in range(len(values_all))}
cat_inds['year_hct'][2020.0] = cat_inds['year_hct'][2019.0]

# get categorical data
X_all_cate = copy.deepcopy(df_train_data[list(cat_inds.keys())])
# map to categorical indices
for tp_var in cat_inds:
    X_all_cate[tp_var] = X_all_cate[tp_var].map(cat_inds[tp_var])
    X_all_cate[tp_var] = X_all_cate[tp_var].astype('category').cat.add_categories(0).fillna(0)
# X_all_cate = X_all_cate.to_numpy(dtype=np.int16)
X_all_num = df_train_data[[tp_var for tp_var in data_type_dict 
                           if tp_var not in cat_inds and tp_var!='efs_time' and tp_var!='efs']] # numerical data
# X_all_num = X_all_num.fillna(-1)

In [4]:
X_all_cate.loc[X_all_cate['hla_high_res_8']==1, 'hla_high_res_8'] = 3
X_all_cate.loc[X_all_cate['hla_high_res_8']==2, 'hla_high_res_8'] = 3

X_all_cate.loc[X_all_cate['hla_low_res_6']==1, 'hla_low_res_6'] = 2

X_all_cate.loc[X_all_cate['hla_high_res_6']==1, 'hla_high_res_6'] = 3
X_all_cate.loc[X_all_cate['hla_high_res_6']==2, 'hla_high_res_6'] = 3

X_all_cate.loc[X_all_cate['hla_high_res_10']==1, 'hla_high_res_10'] = 3
X_all_cate.loc[X_all_cate['hla_high_res_10']==2, 'hla_high_res_10'] = 3

X_all_cate.loc[X_all_cate['hla_match_a_low']==1, 'hla_match_a_low'] = 2

X_all_cate.loc[X_all_cate['karnofsky_score']==1, 'karnofsky_score'] = 2

X_all_cate.loc[X_all_cate['hla_low_res_10']==1, 'hla_low_res_10'] = 2

X_all_cate['age>20'] = (X_all_num['age_at_hct']>20).astype(int)

In [5]:
# fill number nan with mean
X_all_num_fill = X_all_num.to_numpy(dtype=np.float32)
X_all_num_fill[np.isnan(X_all_num_fill)] = np.nanmean(X_all_num_fill)
X_all_num_fill = X_all_num_fill / X_all_num_fill.mean(axis=0).reshape(1, -1)

In [6]:
# convert values to category
X_all_cate_int = X_all_cate.to_numpy(dtype=int)
X_all_cate_one_hot = []
for i in range(X_all_cate_int.shape[1]):
    tp_col = X_all_cate_int[:, i]
    tp_one_hot = np.zeros((tp_col.size, tp_col.max() + 1))
    tp_one_hot[np.arange(tp_col.size), tp_col] = 1
    X_all_cate_one_hot.append(tp_one_hot)
    
X_all_cate_one_hot = np.concatenate(X_all_cate_one_hot, axis=1)

In [7]:
X_all_data = np.concatenate([X_all_num_fill, X_all_cate_one_hot], axis=1)
y_all = copy.deepcopy(df_train_target)
y_all['efs'] = y_all['efs']=='1.0'
y_all = y_all.to_numpy()

In [8]:
# train test split and KNN imputation
train_prop = 0.8
val_prop = 0.2
train_inds = np.random.choice(len(X_all_data), size=int(train_prop*len(X_all_data)), replace=False)
test_inds = [x for x in range(len(X_all_data)) if x not in train_inds]

X_train = X_all_data[train_inds]
X_test = X_all_data[test_inds]

y_train = y_all[train_inds]
y_test = y_all[test_inds]

In [9]:
len(test_inds) + len(train_inds)

28800

# try ML models

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
import math

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [12]:
X_train = torch.tensor(X_train).float().to(device)
X_test = torch.tensor(X_test).float().to(device)

flag_train = torch.tensor([bool(i) for i in y_train[:, 0]]).float().reshape(-1,1).to(device)
flag_test = torch.tensor([bool(i) for i in y_test[:, 0]]).float().reshape(-1,1).to(device)

time_train = torch.tensor([i for i in y_train[:, 1]]).float().reshape(-1,1).to(device)
time_test = torch.tensor([i for i in y_test[:, 1]]).float().reshape(-1,1).to(device)

train_data = torch.utils.data.TensorDataset(X_train, flag_train, time_train)
val_data = torch.utils.data.TensorDataset(X_test, flag_test, time_test)

In [13]:
len(X_train)

23040

In [14]:
def R_set(x):
    '''Create an indicator matrix of risk sets, where T_j >= T_i.
    Note that the input data have been sorted in descending order.
    Input:
        x: a PyTorch tensor that the number of rows is equal to the number of samples.
    Output:
        indicator_matrix: an indicator matrix (which is a lower traiangular portions of matrix).
    '''
    n_sample = x.size(0)
    matrix_ones = torch.ones(n_sample, n_sample)
    indicator_matrix = torch.tril(matrix_ones)

    return(indicator_matrix)

def neg_par_log_likelihood(tp_pred, tp_ytime, tp_yevent):#event=0,censored
    #ytime should be sorted with increasing order
    '''Calculate the average Cox negative partial log-likelihood.
    Input:
        pred: linear predictors from trained model.
        ytime: true survival time from load_data().
        yevent: true censoring status from load_data().
    Output:
        cost: the cost that is to be minimized.
    '''
    # sort pred, ytime, and yevent based on ytime
    sorted_inds = torch.argsort(tp_ytime.reshape(-1), descending=True)
    ytime = tp_ytime[sorted_inds]
    pred = tp_pred[sorted_inds]
#     pred[pred>200] = 200
    yevent = tp_yevent[sorted_inds]
    
    n_observed = yevent.sum(0)
    ytime_indicator = R_set(ytime)
    if torch.cuda.is_available():
        ytime_indicator = ytime_indicator.cuda()
    risk_set_sum = ytime_indicator.mm(torch.exp(pred)) 
    diff = pred - torch.log(risk_set_sum)
    sum_diff_in_observed = torch.transpose(diff, 0, 1).mm(yevent)
    cost = (- (sum_diff_in_observed / n_observed)).reshape((-1,))

    return cost

def get_c_index(pred, flag, time):
    pred = pred.cpu().detach().numpy().reshape(-1)
    flag = flag.cpu().detach().numpy().reshape(-1)>0
    time = time.cpu().detach().numpy().reshape(-1)
    return Cindex(flag, time, pred)[0]

In [15]:
class MPLSampling(nn.Module):
    def __init__(self, config):
        super(MPLSampling, self).__init__()
        self.config = config
        self.n_embed = config.n_embed
        self.n_group = config.n_group
        self.n_model = config.n_model
        
        self.latent = nn.Sequential(
                nn.Linear(config.input_dim, config.n_embed),
                nn.SELU(),
                nn.Linear(config.n_embed, config.n_embed),
                nn.SELU(),
            )
        
        self.MLP = torch.nn.ModuleList([
            nn.Sequential(
                nn.Linear(config.input_dim, config.n_embed),
                nn.SELU(),
                nn.Linear(config.n_embed, config.n_group),
                nn.Softmax(dim=-1),
            )
            for _ in range(config.n_model)
        ])
        
        self.out = nn.Linear(config.n_embed, 1)
        self.dropout = nn.Dropout(config.dropout)
        
        self.sampling_prop = 0.6
        
        self.training_data = None
        self.is_training=True
        
    def initialize_training_data(self, training_data):
        self.training_data = training_data
        self.train_num = training_data.shape[0]
        self.training_inds = []
        for _ in range(self.n_model):
            selected_train_index = np.random.choice(self.train_num, size=int(self.sampling_prop*self.train_num), replace=True)
            self.training_inds.append(selected_train_index)
        
    def forward(self, inputs, targets=None):
        """
        inputs: shape (batch_size, input_dim)
        """
        latent_all = self.latent(self.training_data)
        out = []
        real_out = None
        
        for i in range(self.n_model):
            selected_training_data = self.training_data[self.training_inds[i]]
            latent_vectors = latent_all[self.training_inds[i]] # shape (n_selected, n_embed)
            tp_mlp = self.MLP[i]
            tp_input_gs = tp_mlp(inputs) # shape (batch_size, n_group)
            tp_train_gs = torch.transpose(tp_mlp(selected_training_data), 0,1) # shape (n_group, n_selected)
            
            # construct group representations for the current model
            tp_normalized_training_gs = tp_train_gs / tp_train_gs.sum(axis=1).unsqueeze(-1) # shape (n_group, n_selected)
            tp_group_representation = tp_normalized_training_gs@latent_vectors # shape (n_group, n_embed)
            
            # get vector representation of patients
            tp_patient_vectors = tp_input_gs@tp_group_representation# + input_latent # shape (batch_size, n_embed)
            tp_patient_vectors = self.dropout(tp_patient_vectors)
            tp_out = self.out(tp_patient_vectors) # shape (batch_size, 1)
            
            out.append(tp_out)
            
            if real_out is None:
                real_out = tp_out
            else:
                real_out += tp_out
        
        loss = None
        if targets is not None:
            flags, times = targets
            loss = []
            for tp_out in out:
                tp_loss = neg_par_log_likelihood(tp_out, times, flags)
                loss.append(tp_loss)
                
        return real_out, loss
    
class Config():
    def __init__(self, n_embed, n_layer, n_group, n_model, dropout):
        self.n_embed = n_embed
        self.n_layer = n_layer
        
        self.input_dim = X_train.shape[1]
        self.n_group = n_group
        self.n_model = n_model
        
        self.dropout = dropout

In [17]:
# other parameters
n_embed = 32
n_layer = 2 # number of layers

n_group = 2
n_model = 64 # number of heads
dropout = 0.3

config = Config(n_embed, n_layer, n_group, n_model, dropout)
model = MPLSampling(config)
model.initialize_training_data(X_train)
model.to(device)
batch_size = 4500
max_iter = 200
gamma = 0.5
step_size = 10

lr = 1e-3
tp_lr = lr

tp_ct = 0
best_model = None
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
tp_min_loss = 999999999
past_records = []
lr_count  = 0
print("=> start training")
for epoch in range(1,max_iter+1):
    tp_rec = []
    train_C_index = []
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=9999999, shuffle=True, drop_last=False)
    for tp_X, tp_flag, tp_time in train_loader:
        tp_ct += 1
        model.zero_grad()
        model.is_training=True
        out, loss = model(tp_X, targets=(tp_flag, tp_time))
        
        loss_r = []
        for tp_loss in loss:
            tp_loss.backward(retain_graph=True)
            loss_r.append(tp_loss.cpu().detach())
        optimizer.step()
#         loss_r = loss.cpu().detach()
        tp_rec.extend(loss_r)
        train_C_index.append(get_c_index(out, tp_flag, tp_time))
#         print(loss_r)
    tp_epoch_loss = float(sum(tp_rec)/len(tp_rec))
    tp_epoch_C = float(sum(train_C_index)/len(train_C_index))
    print("current loss: {:.4f}".format(tp_epoch_loss) + " current train C-index: {:.4f}".format(tp_epoch_C))
    if tp_min_loss>tp_epoch_loss: # if the current loss is less than the min_loss record
        tp_min_loss = float(tp_epoch_loss)
        best_model = copy.deepcopy(model)

    # calculate the c-index on validation set
    with torch.no_grad():
        for tp_X, tp_flag, tp_time in val_loader:
            model.is_training=False
            out, loss = model(tp_X, targets=(tp_flag, tp_time))
            tp_c_index = get_c_index(out, tp_flag, tp_time)
            print("c index on validation set: {:.4f}".format(tp_c_index))
            print('current learning rate: {:.6f}'.format(tp_lr))
    print()
    
    scheduler.step()
    if epoch%step_size==0:
        tp_lr *= gamma
    past_records.append(tp_epoch_loss)

=> start training
current loss: 8.0706 current train C-index: 0.5049
c index on validation set: 0.5292
current learning rate: 0.001000

current loss: 8.0662 current train C-index: 0.5442
c index on validation set: 0.5750
current learning rate: 0.001000

current loss: 8.0626 current train C-index: 0.5925
c index on validation set: 0.6221
current learning rate: 0.001000

current loss: 8.0589 current train C-index: 0.6258
c index on validation set: 0.6353
current learning rate: 0.001000

current loss: 8.0552 current train C-index: 0.6392
c index on validation set: 0.6487
current learning rate: 0.001000

current loss: 8.0507 current train C-index: 0.6478
c index on validation set: 0.6536
current learning rate: 0.001000

current loss: 8.0449 current train C-index: 0.6510
c index on validation set: 0.6565
current learning rate: 0.001000

current loss: 8.0365 current train C-index: 0.6527
c index on validation set: 0.6568
current learning rate: 0.001000

current loss: 8.0291 current train C-i

KeyboardInterrupt: 